<a href="https://colab.research.google.com/github/GOKULSINGHSHAH123/Driver-Drowsiness/blob/main/huggingFace.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

In [28]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from transformers import DistilBertTokenizerFast
import tensorflow as tf
from transformers import TFDistilBertForSequenceClassification,TFTrainer,TFTrainingArguments
from sklearn.metrics import confusion_matrix

In [24]:
data = pd.read_csv(r"/content/spam.csv", encoding="ISO-8859-1")


In [25]:
data.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [6]:
df = data.drop(['Unnamed: 2','Unnamed: 3','Unnamed: 4'],axis=1)


In [7]:
df = df.rename(columns={'v1': 'label', 'v2': 'description'})

In [8]:
y = df['label']
X = df['description']

In [9]:
y= y.apply(lambda x:0 if x=='ham' else 1)

In [10]:
X_train,X_test,y_train,y_test = train_test_split(X,y,train_size=0.2,random_state=42)

In [11]:
#!pip install transformers

In [12]:
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

In [13]:
X_train = list(X_train)
X_test = list(X_test)

In [14]:
train_encodings = tokenizer(X_train,truncation=True,padding=True)
test_encodings = tokenizer(X_test,truncation=True,padding=True)

### Converting these encodings to dataset objects

In [15]:
train_dataset = tf.data.Dataset.from_tensor_slices(
    (dict(train_encodings),y_train)
)
test_dataset = tf.data.Dataset.from_tensor_slices(
    (dict(test_encodings),y_test)
)

In [21]:
training_args = TFTrainingArguments(
    output_dir='./reaults',
    num_train_epochs=5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16,
    evaluation_strategy="steps",  # Use "steps" for eval_steps
    eval_steps=500,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
)

In [22]:
with training_args.strategy.scope():
    model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased')

trainer = TFTrainer(
    model = model,
    args = training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

trainer.train()

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_transform.bias']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

In [23]:
trainer.evaluate(test_dataset)

{'eval_loss': 0.05546407460311835}

In [26]:
trainer.predict(test_dataset)

PredictionOutput(predictions=array([[ 3.0449603, -2.9789424],
       [ 3.3894012, -3.352675 ],
       [-2.7487807,  2.9042838],
       ...,
       [ 3.6534693, -3.709271 ],
       [-1.0861263,  1.1833992],
       [ 3.6662612, -3.6646416]], dtype=float32), label_ids=array([0, 0, 1, ..., 0, 0, 0]), metrics={'eval_loss': 0.05638460617338885})

In [29]:
output = trainer.predict(test_dataset)[1]

In [30]:
cm = confusion_matrix(y_test,output)

In [31]:
cm

array([[3868,    0],
       [   0,  590]])